In [25]:
import re,os
import antlr4.tree
from antlr4.tree import Tree
from antlr4.tree.Trees import Trees
from Java8Parser import Java8Parser
from Java8Lexer import Java8Lexer 
from Java8ParserListener import Java8ParserListener
from antlr4 import *
import pickle
import joblib
import numpy as np
import pandas as pd
from glob import glob
import warnings
warnings.filterwarnings("ignore")

## TraverseAST

In [26]:
symbols = ['(',')','{','}','[',']',';',',','.','=','>','<','!','~','?',':','==','<=','>=','!=','&&','||','++','--','+','-','*','/','&','|','^','%','+=','-=','*=','/=','&=','|=','^=','%=','<<=','>>=','>>>=','->','::','@','String','int','char','boolean','double','float','long']
def call(node):
    tem = ''
    if node.getChildCount() !=0:
        for i in range(node.getChildCount()):
            tem += call(node.getChild(i))
    else:
        txt =node.getText()
        if (txt not in symbols):
            txt =" "+txt
        return txt
    if tem!=None:    
        return tem.replace('. ','.')


In [27]:
class LineNumber(Tree.ParseTreeVisitor):
    def __init__(self,line):
        self.code1 =''
        self.code2 =''
        
        self.line = line
        
        
    def doVisit(self, file):
        file = "../Pickle/"+file+'.pkl'
        tree = joblib.load(open(file, "rb"))
        return self.visit(tree)
        
    def visit(self, node):
        
        if node.__class__.__name__ in ['StatementContext' , 'BlockStatementContext']:            
            if node.start.line ==self.line:
                self.code1 = node.getText()
                self.code2 = call(node)
        Tree.ParseTreeVisitor.visit(self, node)
        return self.code1,self.code2


## CSV from log

#### MethodSignature

In [28]:
class listerner(Java8ParserListener):
    def __init__(self,line,no):
        self.line = line
        self.no = no
        self.fun = []
        
    def enterMethodDeclaration(self, ctx:Java8Parser.MethodDeclarationContext):
        if self.line in ctx.getText():
            if ctx.start.line<=self.no and ctx.stop.line>=self.no:
                fun1 = ctx.getChild(1).getText()
                f2 = call(ctx.getChild(2))
                self.fun = fun1+f2
        

In [29]:
class AstProcessor:

    def __init__(self,listener):
        self.listener = listener

    def execute(self, input_source):
        walker = ParseTreeWalker()
        walker.walk(self.listener, input_source)
        return self.listener.fun

In [30]:
def getSourceLine(lineList,fileList):
    sourceline = []
    methodSignature = []
    for line,file in zip(lineList,fileList):
        file = file.replace('.java','')
        if os.path.exists('../pickle/'+file+'.pkl'):
            tree = joblib.load(open('../pickle/'+file+'.pkl', "rb"))
            cLine,orignalline = LineNumber(int(line)).visit(tree)
            sourceline.append(orignalline)
            methodSignature.append(AstProcessor(listerner(cLine,int(line))).execute(tree))
    print(methodSignature)
    sourceline = '\n'.join(sourceline)
    methodSignature = '\n'.join(methodSignature)
    return sourceline,methodSignature

In [33]:
def process(source):
    files = glob(source+"*")
    for path in files:
        
        file = os.path.split(path)[-1].split('.')[0]
        print(file)
        df = pd.DataFrame()
        with open(path,'r') as f:
            txt = f.read()
        pattern = r'((?:java\.|Exception\sin).*\n?(?:.*at.*\)\n)+)'
        exceptionpattern = r'\..*\.(.*?Exception)'
        Exceptioninfopattern = r'.*Exception\:(.*)'
        classPattern = r'at\s(?P<Package>(.*\..*?)*)\.?(?P<Class>(.*))\.(?P<Method>(.*))\((?P<FileName>(.*))\:(?P<LineNo>(\d+))\)'
        df['Exception'] = re.findall(pattern,txt)
        df['ExceptionType'] = df.Exception.str.findall(exceptionpattern)
        df['ExceptionInfo'] = df.Exception.str.findall(Exceptioninfopattern)
        df['Package'] = df.Exception.str.extractall(classPattern).replace(np.nan,'-').groupby(level=0)['Package'].apply(list)
        df['ClassName'] = df.Exception.str.extractall(classPattern).replace(np.nan,'-').groupby(level=0)['Class'].apply(list)
        df['Method'] = df.Exception.str.extractall(classPattern).replace(np.nan,'-').groupby(level=0)['Method'].apply(list)
        df['FileName'] = df.Exception.str.extractall(classPattern).replace(np.nan,'-').groupby(level=0)['FileName'].apply(list)
        df['LineNo'] = df.Exception.str.extractall(classPattern).replace(np.nan,'-').groupby(level=0)['LineNo'].apply(list)
        df[['SourceLine','MethodSignature']] = df[['LineNo','FileName']].apply(lambda x:getSourceLine(x[0],x[1]), axis = 1,result_type='expand')
        df['FileName'] = df['FileName'].apply(lambda x:'\n'.join(x))
        df['LineNo'] = df['LineNo'].apply(lambda x:'\n'.join(x))
        df['ClassName'] = df['ClassName'].apply(lambda x:'\n'.join(x))
        del df['Exception']
        df.to_excel(r'../Data/'+file+'_Exception.xlsx',index =False)
    return df

In [35]:
df = process('../log/')
df

HelloWorld
['test(String arg[],int a,String[] arg)', 'main(String[] args)']
['best(String arg[])', 'main(String[] args)']
['check()', 'main(String[] args)']


ExceptionType                          ExceptionInfo  \
0             [ArithmeticException]                           [ / by zero]   
1  [ArrayIndexOutOfBoundsException]  [ Index 5 out of bounds for length 5]   
2          [InputMismatchException]                                     []   

                                             Package  \
0  [-, -, java.base/jdk.internal.reflect., java.b...   
1  [-, -, java.base/jdk.internal.reflect., java.b...   
2  [java.base/java.util., java.base/java.util., j...   

                                           ClassName  \
0  Fen\nFun\nNativeMethodAccessorImpl\nDelegating...   
1  Fin\nFun\nNativeMethodAccessorImpl\nDelegating...   
2  Scanner\nScanner\nScanner\nScanner\nValidate\n...   

                                              Method  \
0  [<init>, main, invoke, invoke, invoke, execute...   
1  [<init>, main, invoke, invoke, invoke, execute...   
2    [throwFor, next, nextInt, nextInt, check, main]   

                                            FileName  \
0  helloWorld.java\nhelloWorld.java\nNativeMethod...   
1  helloWorld.java\nhelloWorld.java\nNativeMethod...   
2  Scanner.java\nScanner.java\nScanner.java\nScan...   

                               LineNo  \
0   23\n9\n78\n43\n567\n415\n192\n132   
1  42\n10\n78\n43\n567\n415\n192\n132   
2       939\n1594\n2258\n2212\n36\n29   

                                          SourceLine  \
0                     t= 12/ 0;\n Fen fe= new Fen();   
1   System.out.println( a[ i]);\n Fin fi= new Fin();   
2                   int age= s.nextInt();\n check();   

                                     MethodSignature  
0  test(String arg[],int a,String[] arg)\nmain(St...  
1            best(String arg[])\nmain(String[] args)  
2                       check()\nmain(String[] args)